In [189]:
from collections import Counter
import random
import string
import difflib
import time

In [190]:
data = []
vocab = []
total_count = 0
pair_prob = {}
with open('w2_.txt', 'r', encoding='latin-1') as f:
  data = f.readlines()
  for i in data:
    l =  i.rstrip()
    temp = l.split('\t')
    vocab.append(temp[1])
    vocab.append(temp[2])
    pair_prob[f"{temp[1]} {temp[2]}"] = temp[0]
    total_count += int(temp[0])
vocab

['a',
 'a',
 'a',
 'aaa',
 'a',
 'all',
 'a',
 'an',
 'a',
 'and',
 'a',
 'another',
 'a',
 'at',
 'a',
 'b',
 'a',
 'b+',
 'a',
 'b-17',
 'a',
 'b-2',
 'a',
 'b-52',
 'a',
 'b-movie',
 'a',
 'b-plus',
 'a',
 'b.a',
 'a',
 'b.f.a',
 'a',
 'b.s',
 'a',
 'ba',
 'a',
 'babble',
 'a',
 'babbling',
 'a',
 'babe',
 'a',
 'baboon',
 'a',
 'baby',
 'a',
 'baby-faced',
 'a',
 'baby-sitter',
 'a',
 'babysitter',
 'a',
 'babysitting',
 'a',
 'baccalaureate',
 'a',
 'bach',
 'a',
 'bachelor',
 'a',
 'bachelorette',
 'a',
 'bachelors',
 'a',
 'back',
 'a',
 'back-and-forth',
 'a',
 'back-door',
 'a',
 'back-to-basics',
 'a',
 'back-to-school',
 'a',
 'back-up',
 'a',
 'backboard',
 'a',
 'backbone',
 'a',
 'backcountry',
 'a',
 'backdoor',
 'a',
 'backdrop',
 'a',
 'backer',
 'a',
 'backfire',
 'a',
 'background',
 'a',
 'backhand',
 'a',
 'backhanded',
 'a',
 'backhoe',
 'a',
 'backing',
 'a',
 'backlash',
 'a',
 'backlit',
 'a',
 'backlog',
 'a',
 'backpack',
 'a',
 'backpacker',
 'a',
 'backpack

In [191]:
WORDS = Counter(vocab)

In [192]:
for key in pair_prob:
  pair_prob[key] = int(pair_prob[key])/total_count

In [193]:
total_count

286758206

In [194]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [195]:
word = 'lkkes'
t = edits1(word)
for i in t:
  if i in WORDS:
    print(i, WORDS[i])

lakes 48
likes 67


In [196]:
def first_word(word):
  t = edits1(word)
  s = set(edits2(word))
  t = t.union(s)
  if word in WORDS:
    return word
  word = ''
  max = -10
  for i in t:
    if i in WORDS:
      if int(WORDS[i]) > max:
        max = int(WORDS[i])
        word = i
  return word
first_word('we')

'we'

In [197]:
def common_case(word_prev, word):
  t = edits1(word)
  s = edits2(word)
  possible_words = []
  possible_words_2 = []

  final_word = ''
  pair_1 = word_prev + ' ' + word
  if pair_1 in pair_prob:
    return word

  for i in t:
    if i in WORDS:
      possible_words.append(i)
  for i in s:
    if i in WORDS:
      possible_words_2.append(i)

  max = -10
  max_word = -10
  max_word2 = -10
  for word_ in possible_words:

    pair = word_prev +' ' + word_

    if pair in pair_prob:
      if int(pair_prob[pair]) > max:
        max = int(pair_prob[pair])
        final_word = pair.split()[1]
      else:
        if max_word < WORDS[pair.split()[1]]:
          max_word = WORDS[pair.split()[1]]
          final_word = pair.split()[1]

    if final_word == '':
      for word_ in possible_words:
        pair = word_prev +' ' + word_

        if pair in pair_prob:
          if int(pair_prob[pair]) > max:
            max = int(pair_prob[pair])
            final_word = pair.split()[1]
          else:
            if max_word < WORDS[pair.split()[1]]:
              max_word = WORDS[pair.split()[1]]
              final_word = pair.split()[1]
        else:
          if max_word2 < WORDS[word_]:
            max_word2 = WORDS[word_]
            final_word = word_

  return final_word


In [198]:
sentences1 = [
    "The big tree sways in the wind",
    "She runs fast to catch the bus",
    "The cat sleeps on the mat",
    "He eats an apple for lunch",
    "The sun shines brightly in the sky",
    "They play games at the park",
    "I drink water when I'm thirsty",
    "We walk together in the park",
    "She reads books every night",
    "He watches TV after dinner"
]

sentences = []
for sen in sentences1:
    sentences.append(sen.split())
test_sentences = []

for i in range(10):
    for j in range(10):
        new = sentences[i][0].lower()
        for k in range(1, len(sentences[i])):
            new += " " + random.choice(list(edits1(sentences[i][k]))).lower()
        test_sentences.append(new)

In [199]:
# test_sentences

In [200]:
def similarity(s1, s2):
    normalized1 = s1.lower()
    normalized2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
    return matcher.ratio()

In [201]:
print("Test for Dataset with 1 error in word")
count = len(test_sentences)
summator = 0
for k in range(len(test_sentences)):
  w = ''
  prev_word = ''
  for i in range(len(test_sentences[k].split())):
    start = time.time()
    if i == 0:
      w += first_word(test_sentences[k].split()[i]) + ' '
      prev_word = first_word(test_sentences[k].split()[i])
    else:
      w += common_case(prev_word, test_sentences[k].split()[i])+ ' '
      prev_word = common_case(prev_word, test_sentences[k].split()[i])
  print("Sentence before: ", test_sentences[k])
  print("Sentence after: ", w)
  print("Accuracy", similarity( sentences1[k // 10], w))
  print("Time for this sentence", time.time() - start)
  summator += similarity( sentences1[k // 10], w)

print("overall precision", summator/count)

Test for Dataset with 1 error in word
Sentence before:  the xig ltree swvays ii ahe winsd
Sentence after:  the big tree sways in the wins 
Accuracy 0.9508196721311475
Time for this sentence 0.1189429759979248
Sentence before:  the bcig tvree siays iny tde woind
Sentence after:  the big tree says in the wound 
Accuracy 0.9180327868852459
Time for this sentence 0.10155272483825684
Sentence before:  the bikg tdee seways irn th windf
Sentence after:  the big tree sways in th wind 
Accuracy 0.9666666666666667
Time for this sentence 0.10428476333618164
Sentence before:  the bign toee swwys win he wiad
Sentence after:  the big tree sways in he wind 
Accuracy 0.9666666666666667
Time for this sentence 0.06515216827392578
Sentence before:  the bir treie oways ihn tme windn
Sentence after:  the big tree ways in the winds 
Accuracy 0.9508196721311475
Time for this sentence 0.10477209091186523
Sentence before:  the ybig otree swayvs isn tkhe xind
Sentence after:  the big tree sways in the find 
Acc

In [202]:
sentences1 = [
    "The big tree sways in the wind",
    "She runs fast to catch the bus",
    "The cat sleeps on the mat",
    "He eats an apple for lunch",
    "The sun shines brightly in the sky",
    "They play games at the park",
    "I drink water when I'm thirsty",
    "We walk together in the park",
    "She reads books every night",
    "He watches TV after dinner"
]

sentences = []
for sen in sentences1:
    sentences.append(sen.split())
test_sentences = []

for i in range(10):
    for j in range(10):
        new = sentences[i][0].lower()
        for k in range(1, len(sentences[i])):
            new += " " + random.choice(list(edits1(sentences[i][k]))+ list(edits2(sentences[i][k]))).lower()
        test_sentences.append(new)

This a text-correction with 1 error in word. Now we will try to fix words with 2 errors.

In [203]:
print("Test for Dataset with 2 error in word")
count = len(test_sentences)
summator = 0
for k in range(len(test_sentences)):
  w = ''
  prev_word = ''
  for i in range(len(test_sentences[k].split())):
    start = time.time()
    if i == 0:
      w += first_word(test_sentences[k].split()[i]) + ' '
      prev_word = first_word(test_sentences[k].split()[i])
    else:
      w += common_case(prev_word, test_sentences[k].split()[i])+ ' '
      prev_word = common_case(prev_word, test_sentences[k].split()[i])
  print("Sentence before: ", test_sentences[k])
  print("Sentence after: ", w)
  print("Accuracy", similarity( sentences1[k // 10], w))
  print("Time for this sentence", time.time() - start)
  summator += similarity( sentences1[k // 10], w)

print("overall precision", summator/count)


Test for Dataset with 2 error in word
Sentence before:  the xbsig treenk swkpys jvn thyo winpv
Sentence after:  the    jon thy  
Accuracy 0.5217391304347826
Time for this sentence 0.08939933776855469
Sentence before:  the ebgg treog swfawys km tde whqnd
Sentence after:  the egg   am the  
Accuracy 0.5416666666666666
Time for this sentence 0.11400747299194336
Sentence before:  the bivqg fcree swucys min nphe fwnnd
Sentence after:  the  free  in   
Accuracy 0.6086956521739131
Time for this sentence 0.08815860748291016
Sentence before:  the xbigo tweq swayvsk ei thnq wikvd
Sentence after:  the  twea  i   
Accuracy 0.5333333333333333
Time for this sentence 0.11458396911621094
Sentence before:  the bipo torze pswaygs pink thke wwinq
Sentence after:  the biko tore  pick the  
Accuracy 0.6545454545454545
Time for this sentence 0.10199379920959473
Sentence before:  the pipg truxe swayts iz tmle wuhind
Sentence after:  the pipe true sways in tile  
Accuracy 0.7457627118644068
Time for this sent

**Justifying my decision:**

I use Bigrams and my result is following:
~50% accuracy for words with 2 errors (i think it is very good results because some words i cannot recognize by myself)
~90% accuracy for words with 1 error

I refrained from utilizing trigrams and higher-order n-grams due to several considerations:

1. **Sufficiency of Bigrams**: Bigrams are sufficient for understanding context and achieving satisfactory results. This approach is not only effective but also computationally efficient, making it a preferable choice for the task at hand.

2. **Performance and Efficiency**: Bigrams offer a balance between performance and computational resources. While trigrams and higher-order n-grams can provide more detailed context, they come at the cost of increased computational complexity and potentially diminishing returns in terms of accuracy improvements.

3. **Contextual Relevance**: For many applications, the context provided by bigrams is sufficient to make accurate predictions or corrections. Higher-order n-grams may capture more nuanced contexts, but this additional detail is often unnecessary for the task of spelling correction.

4. **Simplicity and Maintainability**: Bigrams simplify the model, making it easier to understand, implement, and maintain. This simplicity is particularly valuable in applications where the model's complexity needs to be kept to a minimum.

In summary, the decision to use bigrams instead of trigrams or higher-order n-grams is based on a combination of factors, including the sufficiency of bigrams for the task, the trade-off between performance and computational efficiency, the relevance of the context captured by bigrams, and the benefits of simplicity and maintainability.

